In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision.models import resnet34
from tqdm import tqdm
import albumentations as A
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 512
batch_size = 4
n_epochs = 10

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
class ForgeryDataset(Dataset):
    def __init__(self, image_paths, mask_paths, is_train=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.is_train = is_train
        self.transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.OneOf([
                A.ImageCompression(quality_lower=30, quality_upper=90, p=1.0),
                A.GaussianBlur(blur_limit=(3, 7), p=1.0),
            ], p=0.5)
        ]) if is_train else None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            image = cv2.imread(self.image_paths[idx])[..., ::-1]  # BGR to RGB
            mask = cv2.imread(self.mask_paths[idx], 0)  # Grayscale

            # Resize if needed
            image = cv2.resize(image, (image_size, image_size))
            mask = cv2.resize(mask, (image_size, image_size))

            if self.transform and self.is_train:
                augmented = self.transform(image=image, mask=mask)
                image, mask = augmented['image'], augmented['mask']

            # Normalize and convert to tensor
            image = torch.FloatTensor(image.transpose(2, 0, 1) / 255.)
            mask = torch.FloatTensor(mask / 255.).unsqueeze(0)

            return image, mask

        except Exception as e:
            print(f"Error loading {self.image_paths[idx]}: {e}")
            dummy_image = torch.zeros((3, image_size, image_size))
            dummy_mask = torch.zeros((1, image_size, image_size))
            return dummy_image, dummy_mask

In [ ]:
import os
import numpy as np

# Define paths
base_dir = "/content/drive/MyDrive/project/ImageForensicsOSN/data/ImageForgeriesOSN_Dataset"
output_file = "/content/drive/MyDrive/project/ImageForensicsOSN/data/traindso.npy"

# Initialize lists to store image and mask paths
image_paths = []
mask_paths = []

# Define dataset
dataset = "dso"

# Corrected folder names
forgeries_dir = os.path.join(base_dir, dataset, "DSO_Facebook")
masks_dir = os.path.join(base_dir, dataset, "DSO_GT")

# Supported image extensions
image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"]
mask_extensions = [".png", ".tif", ".tiff"]

# Check if directories exist
if not os.path.exists(forgeries_dir):
    print(f"Error: Forgeries directory '{forgeries_dir}' not found!")
if not os.path.exists(masks_dir):
    print(f"Error: Masks directory '{masks_dir}' not found!")

# Load images and masks
if os.path.exists(forgeries_dir) and os.path.exists(masks_dir):
    for img_name in os.listdir(forgeries_dir):
        img_ext = os.path.splitext(img_name)[1].lower()
        if img_ext not in image_extensions:
            continue  # Skip unsupported formats

        # Construct image and mask paths
        img_path = os.path.join(forgeries_dir, img_name)
        mask_name = os.path.splitext(img_name)[0] + "_gt"
        mask_path = None

        # Look for a mask with a supported extension
        for ext in mask_extensions:
            potential_mask_path = os.path.join(masks_dir, mask_name + ext)
            if os.path.exists(potential_mask_path):
                mask_path = potential_mask_path
                break

        if mask_path:
            image_paths.append(img_path)
            mask_paths.append(mask_path)

# Combine image and mask paths into pairs
train_data = list(zip(image_paths, mask_paths))

# Save to .npy file
np.save(output_file, train_data)

print(f"traindso.npy file created at {output_file}")
print(f"Total samples: {len(train_data)}")


traindso.npy file created at /content/drive/MyDrive/project/ImageForensicsOSN/data/traindso.npy
Total samples: 100


In [ ]:
class ForgeryDetector(nn.Module):
    def __init__(self):
        super().__init__()
        # Use ResNet34 as backbone
        self.backbone = resnet34(weights='DEFAULT')

        # Decoder with proper upsampling to 512x512
        self.decoder = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Encoder
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # Decoder
        x = self.decoder(x)
        return x

In [ ]:
def calculate_metrics(pred, target):
    pred = pred.flatten()
    target = target.flatten()

    # Binarize
    pred_bin = (pred > 0.5).astype(np.float32)
    target_bin = (target > 0.5).astype(np.float32)

    # Calculate metrics
    tp = np.sum((pred_bin == 1) & (target_bin == 1))
    fp = np.sum((pred_bin == 1) & (target_bin == 0))
    fn = np.sum((pred_bin == 0) & (target_bin == 1))
    tn = np.sum((pred_bin == 0) & (target_bin == 0))

    # Avoid division by zero
    epsilon = 1e-7

    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn + epsilon)

    # Precision
    precision = tp / (tp + fp + epsilon)

    # Recall
    recall = tp / (tp + fn + epsilon)

    # F1 Score
    f1 = 2 * (precision * recall) / (precision + recall + epsilon)

    # IoU
    iou = tp / (tp + fp + fn + epsilon)

    # AUC
    if len(np.unique(target_bin)) > 1:
        auc = roc_auc_score(target_bin, pred)
    else:
        auc = 0.5

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'iou': iou,
        'auc': auc
    }

In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, masks in tqdm(loader, desc="Training"):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Validating"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(masks.cpu().numpy())

    # Concatenate all batches
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)

    return calculate_metrics(preds, targets)

def train_model():
    # Load data
    data = np.load('/content/drive/MyDrive/project/ImageForensicsOSN/data/train.npy', allow_pickle=True)
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    train_images = [item[0] for item in train_data]
    train_masks = [item[1] for item in train_data]
    val_images = [item[0] for item in val_data]
    val_masks = [item[1] for item in val_data]

    # Create datasets and loaders
    train_dataset = ForgeryDataset(train_images, train_masks, is_train=True)
    val_dataset = ForgeryDataset(val_images, val_masks, is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = ForgeryDetector().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
    criterion = nn.BCELoss()

    best_metrics = {'f1': 0}
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")

        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion)

        # Validate
        metrics = validate(model, val_loader)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Metrics:")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1: {metrics['f1']:.4f}")
        print(f"  IoU: {metrics['iou']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")

        # Save best model
        if metrics['f1'] > best_metrics['f1']:
            best_metrics = metrics
            torch.save(model.state_dict(), "/content/drive/MyDrive/project/ImageForensicsOSN/best_model.pth")
            print(f"New best model saved with F1: {metrics['f1']:.4f}")

    print("\nTraining complete!")
    print("Best Validation Metrics:")
    for k, v in best_metrics.items():
        print(f"  {k}: {v:.4f}")

In [ ]:
# Start training
train_model()

<ipython-input-5-428c9884c28e>:9: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=90, p=1.0),
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 189MB/s]



Epoch 1/10


Validating: 100%|██████████| 29/29 [02:31<00:00,  5.22s/it]


Train Loss: 0.2674
Val Metrics:
  AUC: 0.7970
  Accuracy: 0.9208
  F1: 0.0003
  IoU: 0.0001
  Precision: 0.8020
  Recall: 0.0001
New best model saved with F1: 0.0003

Epoch 2/10


Validating: 100%|██████████| 29/29 [00:20<00:00,  1.44it/s]


...

Train Loss: 0.0763
Val Metrics:
  AUC: 0.9556
  Accuracy: 0.9609
  F1: 0.7177
  IoU: 0.5597
  Precision: 0.8391
  Recall: 0.6271
New best model saved with F1: 0.7177

Epoch 9/10


Validating: 100%|██████████| 29/29 [00:20<00:00,  1.39it/s]


Train Loss: 0.0698
Val Metrics:
  AUC: 0.9591
  Accuracy: 0.9638
  F1: 0.7446
  IoU: 0.5931
  Precision: 0.8442
  Recall: 0.6660
New best model saved with F1: 0.7446

Epoch 10/10


Validating: 100%|██████████| 29/29 [00:20<00:00,  1.39it/s]


Train Loss: 0.0676
Val Metrics:
  AUC: 0.9604
  Accuracy: 0.9635
  F1: 0.7414
  IoU: 0.5890
  Precision: 0.8442
  Recall: 0.6609

Training complete!
Best Validation Metrics:
  accuracy: 0.9638
  precision: 0.8442
  recall: 0.6660
  f1: 0.7446
  iou: 0.5931
  auc: 0.9591


In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, masks in tqdm(loader, desc="Training"):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Validating"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(masks.cpu().numpy())

    # Concatenate all batches
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)

    return calculate_metrics(preds, targets)

def train_model2():
    # Load data
    data = np.load('/content/drive/MyDrive/project/ImageForensicsOSN/data/traincasia.npy', allow_pickle=True)
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    train_images = [item[0] for item in train_data]
    train_masks = [item[1] for item in train_data]
    val_images = [item[0] for item in val_data]
    val_masks = [item[1] for item in val_data]

    # Create datasets and loaders
    train_dataset = ForgeryDataset(train_images, train_masks, is_train=True)
    val_dataset = ForgeryDataset(val_images, val_masks, is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = ForgeryDetector().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
    criterion = nn.BCELoss()

    best_metrics = {'f1': 0}
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")

        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion)

        # Validate
        metrics = validate(model, val_loader)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Metrics:")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1: {metrics['f1']:.4f}")
        print(f"  IoU: {metrics['iou']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")

        # Save best model
        if metrics['f1'] > best_metrics['f1']:
            best_metrics = metrics
            torch.save(model.state_dict(), "/content/drive/MyDrive/project/ImageForensicsOSN/best_modelcasia.pth")
            print(f"New best model saved with F1: {metrics['f1']:.4f}")

    print("\nTraining complete!")
    print("Best Validation Metrics:")
    for k, v in best_metrics.items():
        print(f"  {k}: {v:.4f}")

In [ ]:
train_model2()

<ipython-input-22-428c9884c28e>:9: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=90, p=1.0),



Epoch 1/10


Validating: 100%|██████████| 46/46 [02:32<00:00,  3.31s/it]


Train Loss: 0.2712
Val Metrics:
  AUC: 0.8134
  Accuracy: 0.9109
  F1: 0.2205
  IoU: 0.1239
  Precision: 0.6459
  Recall: 0.1330
New best model saved with F1: 0.2205

Epoch 2/10


Validating: 100%|██████████| 46/46 [00:04<00:00,  9.54it/s]


...

Train Loss: 0.0558
Val Metrics:
  AUC: 0.9089
  Accuracy: 0.9366
  F1: 0.6258
  IoU: 0.4554
  Precision: 0.7096
  Recall: 0.5597

Epoch 9/10


Validating: 100%|██████████| 46/46 [00:04<00:00,  9.34it/s]


Train Loss: 0.0512
Val Metrics:
  AUC: 0.9215
  Accuracy: 0.9403
  F1: 0.6501
  IoU: 0.4816
  Precision: 0.7306
  Recall: 0.5856

Epoch 10/10


Validating: 100%|██████████| 46/46 [00:05<00:00,  8.96it/s]


Train Loss: 0.0468
Val Metrics:
  AUC: 0.9223
  Accuracy: 0.9409
  F1: 0.6385
  IoU: 0.4690
  Precision: 0.7593
  Recall: 0.5509

Training complete!
Best Validation Metrics:
  accuracy: 0.9419
  precision: 0.7510
  recall: 0.5785
  f1: 0.6536
  iou: 0.4854
  auc: 0.9149


In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, masks in tqdm(loader, desc="Training"):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Validating"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(masks.cpu().numpy())

    # Concatenate all batches
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)

    return calculate_metrics(preds, targets)

def train_model3():
    # Load data
    data = np.load('/content/drive/MyDrive/project/ImageForensicsOSN/data/traincolumbia.npy', allow_pickle=True)
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    train_images = [item[0] for item in train_data]
    train_masks = [item[1] for item in train_data]
    val_images = [item[0] for item in val_data]
    val_masks = [item[1] for item in val_data]

    # Create datasets and loaders
    train_dataset = ForgeryDataset(train_images, train_masks, is_train=True)
    val_dataset = ForgeryDataset(val_images, val_masks, is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = ForgeryDetector().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
    criterion = nn.BCELoss()

    best_metrics = {'f1': 0}
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")

        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion)

        # Validate
        metrics = validate(model, val_loader)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Metrics:")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1: {metrics['f1']:.4f}")
        print(f"  IoU: {metrics['iou']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")

        # Save best model
        if metrics['f1'] > best_metrics['f1']:
            best_metrics = metrics
            torch.save(model.state_dict(), "/content/drive/MyDrive/project/ImageForensicsOSN/best_modelcolumbia.pth")
            print(f"New best model saved with F1: {metrics['f1']:.4f}")

    print("\nTraining complete!")
    print("Best Validation Metrics:")
    for k, v in best_metrics.items():
        print(f"  {k}: {v:.4f}")

In [ ]:
train_model3()

<ipython-input-4-428c9884c28e>:9: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=90, p=1.0),
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 198MB/s]



Epoch 1/10


Validating: 100%|██████████| 8/8 [00:35<00:00,  4.47s/it]


Train Loss: 0.4691
Val Metrics:
  AUC: 0.8693
  Accuracy: 0.8183
  F1: 0.6495
  IoU: 0.4809
  Precision: 0.6321
  Recall: 0.6678
New best model saved with F1: 0.6495

Epoch 2/10


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.20it/s]


...

Train Loss: 0.0463
Val Metrics:
  AUC: 0.9543
  Accuracy: 0.8964
  F1: 0.8045
  IoU: 0.6730
  Precision: 0.7673
  Recall: 0.8456
New best model saved with F1: 0.8045

Epoch 9/10


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.13it/s]


Train Loss: 0.0394
Val Metrics:
  AUC: 0.9546
  Accuracy: 0.8880
  F1: 0.7949
  IoU: 0.6597
  Precision: 0.7381
  Recall: 0.8613

Epoch 10/10


Validating: 100%|██████████| 8/8 [00:01<00:00,  6.11it/s]


Train Loss: 0.0403
Val Metrics:
  AUC: 0.9542
  Accuracy: 0.8903
  F1: 0.7973
  IoU: 0.6630
  Precision: 0.7464
  Recall: 0.8558

Training complete!
Best Validation Metrics:
  accuracy: 0.8964
  precision: 0.7673
  recall: 0.8456
  f1: 0.8045
  iou: 0.6730
  auc: 0.9543


In [ ]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, masks in tqdm(loader, desc="Training"):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def validate(model, loader):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in tqdm(loader, desc="Validating"):
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            all_preds.append(outputs.cpu().numpy())
            all_targets.append(masks.cpu().numpy())

    # Concatenate all batches
    preds = np.concatenate(all_preds)
    targets = np.concatenate(all_targets)

    return calculate_metrics(preds, targets)

def train_model4():
    # Load data
    data = np.load('/content/drive/MyDrive/project/ImageForensicsOSN/data/traindso.npy', allow_pickle=True)
    train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

    train_images = [item[0] for item in train_data]
    train_masks = [item[1] for item in train_data]
    val_images = [item[0] for item in val_data]
    val_masks = [item[1] for item in val_data]

    # Create datasets and loaders
    train_dataset = ForgeryDataset(train_images, train_masks, is_train=True)
    val_dataset = ForgeryDataset(val_images, val_masks, is_train=False)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = ForgeryDetector().to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
    criterion = nn.BCELoss()

    best_metrics = {'f1': 0}
    for epoch in range(n_epochs):
        print(f"\nEpoch {epoch+1}/{n_epochs}")

        # Train
        train_loss = train_epoch(model, train_loader, optimizer, criterion)

        # Validate
        metrics = validate(model, val_loader)
        scheduler.step()

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Metrics:")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1: {metrics['f1']:.4f}")
        print(f"  IoU: {metrics['iou']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")

        # Save best model
        if metrics['f1'] > best_metrics['f1']:
            best_metrics = metrics
            torch.save(model.state_dict(), "/content/drive/MyDrive/project/ImageForensicsOSN/best_modeldso.pth")
            print(f"New best model saved with F1: {metrics['f1']:.4f}")

    print("\nTraining complete!")
    print("Best Validation Metrics:")
    for k, v in best_metrics.items():
        print(f"  {k}: {v:.4f}")

In [ ]:
train_model4()

<ipython-input-4-428c9884c28e>:9: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=30, quality_upper=90, p=1.0),



Epoch 1/10


Validating: 100%|██████████| 5/5 [00:29<00:00,  5.84s/it]


Train Loss: 0.4088
Val Metrics:
  AUC: 0.8141
  Accuracy: 0.8600
  F1: 0.0000
  IoU: 0.0000
  Precision: 0.0000
  Recall: 0.0000

Epoch 2/10


Validating: 100%|██████████| 5/5 [00:03<00:00,  1.57it/s]


...

Train Loss: 0.0746
Val Metrics:
  AUC: 0.8549
  Accuracy: 0.8770
  F1: 0.4684
  IoU: 0.3058
  Precision: 0.5931
  Recall: 0.3870

Epoch 9/10


Validating: 100%|██████████| 5/5 [00:03<00:00,  1.47it/s]


Train Loss: 0.0671
Val Metrics:
  AUC: 0.8571
  Accuracy: 0.8758
  F1: 0.4729
  IoU: 0.3096
  Precision: 0.5827
  Recall: 0.3979

Epoch 10/10


Validating: 100%|██████████| 5/5 [00:02<00:00,  1.93it/s]


Train Loss: 0.0679
Val Metrics:
  AUC: 0.8568
  Accuracy: 0.8743
  F1: 0.4836
  IoU: 0.3189
  Precision: 0.5691
  Recall: 0.4204

Training complete!
Best Validation Metrics:
  accuracy: 0.8717
  precision: 0.5501
  recall: 0.4578
  f1: 0.4997
  iou: 0.3331
  auc: 0.8641


In [3]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet34
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Config - should match training config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 512
batch_size = 4

class ForgeryDataset(Dataset):
    def __init__(self, image_paths, mask_paths=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.has_masks = mask_paths is not None

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            image = cv2.imread(self.image_paths[idx])[..., ::-1]  # BGR to RGB
            image = cv2.resize(image, (image_size, image_size))

            # Normalize and convert to tensor
            image = torch.FloatTensor(image.transpose(2, 0, 1) / 255)

            if self.has_masks:
                mask = cv2.imread(self.mask_paths[idx], 0)  # Grayscale
                mask = cv2.resize(mask, (image_size, image_size))
                # Convert to binary mask (0 or 1)
                _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
                mask = torch.FloatTensor(mask / 255.).unsqueeze(0)
                return image, mask, self.image_paths[idx]
            else:
                return image, self.image_paths[idx]

        except Exception as e:
            print(f"Error loading {self.image_paths[idx]}: {e}")
            if self.has_masks:
                dummy_image = torch.zeros((3, image_size, image_size))
                dummy_mask = torch.zeros((1, image_size, image_size))
                return dummy_image, dummy_mask, self.image_paths[idx]
            else:
                dummy_image = torch.zeros((3, image_size, image_size))
                return dummy_image, self.image_paths[idx]

class ForgeryDetector(nn.Module):
    def __init__(self):
        super().__init__()
        # Use ResNet34 as backbone
        self.backbone = resnet34(weights=None)  # Weights will be loaded from state_dict

        # Decoder with proper upsampling to 512x512
        self.decoder = nn.Sequential(
            nn.Conv2d(512, 256, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(256, 128, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(32, 16, 3, padding=1),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(16, 1, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Encoder
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)
        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        # Decoder
        x = self.decoder(x)
        return x

def calculate_metrics(pred, target):
    pred = pred.flatten()
    target = target.flatten()

    # Binarize predictions
    pred_bin = (pred > 0.5).astype(np.float32)
    # Target is already binary (0 or 1)
    target_bin = target.astype(np.float32)

    # Calculate metrics
    tp = np.sum((pred_bin == 1) & (target_bin == 1))
    fp = np.sum((pred_bin == 1) & (target_bin == 0))
    fn = np.sum((pred_bin == 0) & (target_bin == 1))
    tn = np.sum((pred_bin == 0) & (target_bin == 0))

    # Avoid division by zero
    epsilon = 1e-7

    # Accuracy
    accuracy = (tp + tn) / (tp + tn + fp + fn + epsilon)

    # Precision
    precision = tp / (tp + fp + epsilon)

    # Recall
    recall = tp / (tp + fn + epsilon)

    # F1 Score
    f1 = 2 * (precision * recall) / (precision + recall + epsilon)

    # IoU
    iou = tp / (tp + fp + fn + epsilon)

    # AUC
    if len(np.unique(target_bin)) > 1:
        auc = roc_auc_score(target_bin, pred)
    else:
        auc = 0.5

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'iou': iou,
        'auc': auc
    }

def visualize_results(image, pred_mask, true_mask=None, save_path=None):
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis('off')

    # Predicted Mask (show as binary)
    plt.subplot(1, 3, 2)
    plt.imshow((pred_mask > 0.5).astype(np.uint8), cmap='gray')
    plt.title("Predicted Binary Mask")
    plt.axis('off')

    # True Mask (if available, show as binary)
    if true_mask is not None:
        plt.subplot(1, 3, 3)
        plt.imshow(true_mask, cmap='gray')
        plt.title("True Binary Mask")
        plt.axis('off')

    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=300)
    else:
        plt.show()
    plt.close()

def test_model(model_path, test_images, test_masks=None, output_dir='results'):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load model
    model = ForgeryDetector().to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    # Create dataset and loader
    has_masks = test_masks is not None
    test_dataset = ForgeryDataset(test_images, test_masks)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    all_preds = []
    all_targets = []
    all_image_paths = []

    with torch.no_grad():
        for batch in test_loader:
            if has_masks:
                images, masks, paths = batch
                images, masks = images.to(device), masks.to(device)
                all_targets.append(masks.cpu().numpy())
            else:
                images, paths = batch
                images = images.to(device)

            outputs = model(images)
            all_preds.append(outputs.cpu().numpy())
            all_image_paths.extend(paths)

    # Process results
    preds = np.concatenate(all_preds)

    # Save and visualize results
    for i in range(len(preds)):
        image_path = all_image_paths[i]
        image_name = os.path.basename(image_path)

        # Get original image
        orig_image = cv2.imread(image_path)[..., ::-1]
        orig_image = cv2.resize(orig_image, (image_size, image_size))

        # Get predicted mask
        pred_mask = preds[i][0]  # Remove batch and channel dim

        # Get true mask if available
        true_mask = None
        if has_masks:
            true_mask = all_targets[i][0]

        # Visualize and save
        save_path = os.path.join(output_dir, f"result_{image_name}")
        visualize_results(orig_image, pred_mask, true_mask, save_path)

    # Calculate metrics if masks are available
    if has_masks:
        targets = np.concatenate(all_targets)
        metrics = calculate_metrics(preds, targets)

        print("\nTest Metrics:")
        print(f"  AUC: {metrics['auc']:.4f}")
        print(f"  Accuracy: {metrics['accuracy']:.4f}")
        print(f"  F1: {metrics['f1']:.4f}")
        print(f"  IoU: {metrics['iou']:.4f}")
        print(f"  Precision: {metrics['precision']:.4f}")
        print(f"  Recall: {metrics['recall']:.4f}")

        return metrics
    else:
        print(f"\nInference complete! Results saved to {output_dir}")
        return None

if __name__ == "__main__":
    # Load your test data
    test_images = ["/content/drive/MyDrive/project/ImageForensicsOSN/data/ImageForgeriesOSN_Dataset/nist16/NIST16_Facebook/000001.jpg"]
    test_masks = None

    # Test the model
    test_model(
        model_path="/content/drive/MyDrive/project/ImageForensicsOSN/best_model.pth",  # Path to your saved model
        test_images=test_images,
        test_masks=test_masks,
        output_dir="/content/drive/MyDrive/project/ImageForensicsOSN/test_results"
    )


Inference complete! Results saved to /content/drive/MyDrive/project/ImageForensicsOSN/test_results
